In [31]:
from dotenv import load_dotenv
import boto3
import os
import pymysql
import pandas as pd
import chardet
# Database connection parameters
# load_dotenv("C:\\Users\\TalAb\\OneDrive - Elad Software Systems\\Desktop\\super_market_project\\super_market_project\\.env")
# load_dotenv("C:\\Users\\TalAb\\OneDrive - Elad Software Systems\\Desktop\\Colmobil_POC\\colmobil-chatbot-project\\.env")

# endpoint = os.getenv('RDS_ENDPOINT')
# username = os.getenv('RDS_USERNAME')
# password = os.getenv('RDS_PASSWORD')
# database = os.getenv('RDS_DATABASE')
port = 3306

endpoint="database-colmobil.c9owiq2sebpi.us-east-1.rds.amazonaws.com"
username="admin"
password="Bb123456!"
database="databasecolmobil"
# Establish a connection to the RDS instance
connection = pymysql.connect(
    host=endpoint,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()
# Execute a query
cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print(f"Database version: {version}")

Database version: ('8.0.39',)


In [32]:
df = pd.read_excel("colmobil (1).xlsx")

In [33]:
list(df.columns)

[0,
 'Has Sunroof',
 'Ground Clearance(mm)',
 'Charging time (Minutes)',
 'Num. of doors',
 'additional_description',
 'Car Type',
 'Model',
 'Brand',
 'Creation Year',
 'Engine Type',
 'Engine Displacement(KW) ',
 'Engine Displacement (Liter)',
 'Horsepower',
 'Torque (Nm)',
 'Drive Type',
 'Gearbox type',
 'Num. of gears',
 'Acceleration 0 to 100 km/h',
 'Top Speed',
 'Average Energy Consumption (kWh per 100 km)',
 'Combined Fuel Consumption (liters per 100 km)',
 'Electric Driving Range (km)',
 'Battery Capacity (kWh)',
 'Trunk capacity (liters)',
 'Num. of Seats ',
 'Fuel type ',
 'Length (mm)',
 'Width (mm)',
 'Height (mm)',
 'Wheelbase(mm)',
 'Curb Weight (kg)',
 'Basic price (NIS)',
 'Safety System(type)',
 'Multimedia System(type)',
 'Screen Size',
 'Wheel Rim Size (inch)-max',
 'Wheel rim size (inch)-min',
 'Front Lighting (type)',
 'Climate Control System (type)',
 'Vehicle Warranty (years)',
 'Battery Warranty (Years)',
 'Image URL']

In [34]:
print(endpoint)

database-colmobil.c9owiq2sebpi.us-east-1.rds.amazonaws.com


In [40]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

# Get all table names
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

# Drop each table
for (table_name,) in tables:
    drop_table_sql = f"DROP TABLE IF EXISTS {table_name}"
    cursor.execute(drop_table_sql)
    print(f"Dropped table {table_name}")

# Re-enable foreign key checks
cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

Dropped table cars_collection


0

cars_collection table creation

In [41]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS cars_collection (
    car_id INT AUTO_INCREMENT PRIMARY KEY,
    has_sunroof BOOLEAN,
    ground_clearance_mm INT,
    charging_time_minutes INT,
    num_of_doors INT,
    additional_description TEXT,
    embeddings_vector_additional_description BLOB,
    car_type ENUM('SUV', 'Electric SUV', 'Sedan', 'Hatchback', 'Electric Sedan', 'Executive Coupe Sedan'),
    model VARCHAR(50),
    embeddings_vector_model BLOB,
    brand ENUM('Hyundai', 'Ora', 'Chery', 'Mitsubishi', 'Smart'),
    creation_year INT,
    engine_type ENUM('Petrol', 'EV', 'HEV', 'MHEV'),
    engine_displacement_kw DECIMAL(5, 1),
    engine_displacement_liters DECIMAL(3, 1),
    horsepower INT,
    torque_nm INT,
    drive_type ENUM('AWD', 'FWD', 'RWD', '4WD'),
    gearbox_type ENUM('Auto', 'Manual'),
    num_of_gears INT,
    acceleration_0_100 DECIMAL(4, 2),
    top_speed_kmh INT,
    avg_energy_consumption_kwh DECIMAL(4, 2),
    combined_fuel_consumption_liters DECIMAL(4, 2),
    electric_driving_range_km INT,
    battery_capacity_kwh DECIMAL(5, 2),
    trunk_capacity_liters INT,
    num_of_seats INT,
    fuel_type ENUM('Petrol', 'Electric', 'Hybrid'),
    length_mm INT,
    width_mm INT,
    height_mm INT,
    wheelbase_mm INT,
    curb_weight_kg INT,
    basic_price_nis INT,
    safety_system_type ENUM('Advanced', 'Basic'),
    multimedia_system_type ENUM('Advanced', 'Basic'),
    screen_size DECIMAL(4, 2),
    wheel_rim_size_max INT,
    wheel_rim_size_min INT,
    front_lighting_type ENUM('LED', 'Halogen'),
    climate_control_system_type ENUM('Digital', 'Manual'),
    vehicle_warranty_years INT,
    battery_warranty_years INT,
    image_url VARCHAR(255)
);
"""

# Execute the SQL statement
cursor.execute(create_table_sql)

0

recipes table creation

In [42]:
import pandas as pd


def handle_nan(value):
    if pd.isna(value):
        return "NULL"  # Return NULL for NaN values
    elif isinstance(value, str):
        # Escape special characters (like single quotes) for SQL
        return f"'{value.replace("'", "''")}'"
    else:
        return value



for index, row in df.iterrows():
    

    # Prepare the insert statement with NULL values for NaN fields
    insert_cars_collection = f"""  
    INSERT INTO cars_collection (
        has_sunroof, ground_clearance_mm, charging_time_minutes, num_of_doors, additional_description, embeddings_vector_additional_description,
        car_type, model,embeddings_vector_model, brand, creation_year, engine_type, engine_displacement_kw, engine_displacement_liters,
        horsepower, torque_nm, drive_type, gearbox_type, num_of_gears, acceleration_0_100, top_speed_kmh,
        avg_energy_consumption_kwh, combined_fuel_consumption_liters, electric_driving_range_km,
        battery_capacity_kwh, trunk_capacity_liters, num_of_seats, fuel_type, length_mm, width_mm, height_mm,
        wheelbase_mm, curb_weight_kg, basic_price_nis, safety_system_type, multimedia_system_type,
        screen_size, wheel_rim_size_max, wheel_rim_size_min, front_lighting_type, climate_control_system_type,
        vehicle_warranty_years, battery_warranty_years, image_url
    ) VALUES (
    {handle_nan(row['Has Sunroof'])},
    {handle_nan(row['Ground Clearance(mm)'])},
    {handle_nan(row['Charging time (Minutes)'])},
    {handle_nan(row['Num. of doors'])},
    {handle_nan(row['additional_description'])},
    NULL,
    {handle_nan(row['Car Type'])},
    {handle_nan(row['Model'])},
    NULL,
    {handle_nan(row['Brand'])},
    {handle_nan(row['Creation Year'])},
    {handle_nan(row['Engine Type'])},
    {handle_nan(row['Engine Displacement(KW) '])},
    {handle_nan(row['Engine Displacement (Liter)'])},
    {handle_nan(row['Horsepower'])},
    {handle_nan(row['Torque (Nm)'])},
    {handle_nan(row['Drive Type'])},
    {handle_nan(row['Gearbox type'])},
    {handle_nan(row['Num. of gears'])},
    {handle_nan(row['Acceleration 0 to 100 km/h'])},
    {handle_nan(row['Top Speed'])},
    {handle_nan(row['Average Energy Consumption (kWh per 100 km)'])},
    {handle_nan(row['Combined Fuel Consumption (liters per 100 km)'])},
    {handle_nan(row['Electric Driving Range (km)'])},
    {handle_nan(row['Battery Capacity (kWh)'])},
    {handle_nan(row['Trunk capacity (liters)'])},
    {handle_nan(row['Num. of Seats '])},
    {handle_nan(row['Fuel type '])},
    {handle_nan(row['Length (mm)'])},
    {handle_nan(row['Width (mm)'])},
    {handle_nan(row['Height (mm)'])},
    {handle_nan(row['Wheelbase(mm)'])},
    {handle_nan(row['Curb Weight (kg)'])},
    {handle_nan(row['Basic price (NIS)'])},
    {handle_nan(row['Safety System(type)'])},
    {handle_nan(row['Multimedia System(type)'])},
    {handle_nan(row['Screen Size'])},
    {handle_nan(row['Wheel Rim Size (inch)-max'])},
    {handle_nan(row['Wheel rim size (inch)-min'])},
    {handle_nan(row['Front Lighting (type)'])},
    {handle_nan(row['Climate Control System (type)'])},
    {handle_nan(row['Vehicle Warranty (years)'])},
    {handle_nan(row['Battery Warranty (Years)'])},
    {handle_nan(row['Image URL'])}
    )"""  

    cursor.execute(insert_cars_collection)

In [43]:
cursor.execute("SELECT * FROM cars_collection")

# Fetch all rows
rows = cursor.fetchall()

# Optionally, fetch the column names
columns = [description[0] for description in cursor.description]

In [44]:
sql_df = pd.DataFrame(rows, columns=columns)

# Display the data in a tabular format
print(sql_df.head())  # Display the first few rows
print(sql_df)

   car_id  has_sunroof  ground_clearance_mm  charging_time_minutes  \
0       1            1                  170                    NaN   
1       2            1                  185                    NaN   
2       3            1                  173                    NaN   
3       4            1                  155                  422.0   
4       5            1                  135                    NaN   

   num_of_doors                             additional_description  \
0             4  רכב שטח אידיאלי למשפחה שמחפשת את השילוב המושלם...   
1             5  רכב שטח פרימיום שמפגיש בין עיצוב מרהיב לנוחות ...   
2             5  רכב שטח קומפקטי שמציע עיצוב צעיר ודינמי, אידיא...   
3             5  רכב חשמלי מתקדם עם עיצוב עתידני וטכנולוגיה חדש...   
4             4  רכב משפחתי עם עיצוב נקי ומרווח פנימי שמספק נוח...   

  embeddings_vector_additional_description      car_type     model  \
0                                     None           SUV    Tucson   
1                 

In [45]:

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()